In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import ast

# Load your dataset
df = pd.read_csv('final_merged_sheet.csv')
print("Initial shape:", df.shape)
print("Initial columns:", df.columns.tolist())

# --- 1. DROP UNNECESSARY COLUMNS ---
cols_to_drop = [
    'check_in_time', 'check_out_time', 'phone_number', 'email', 
    'website', 'booking_url', 'data_source', 'address', 
    'Distance_to_Landmark'  # Drop text version, keep Distance_to_Landmark(in_km)
]
df_clean = df.drop(columns=[col for col in cols_to_drop if col in df.columns])
print(f"After dropping columns: {df_clean.shape}")

# --- 2. CLEAN CITY AND STATE COLUMNS ---
# Fill missing city/state values
df_clean['city'] = df_clean['city'].fillna('Unknown')
df_clean['state'] = df_clean['state'].fillna('Unknown')

# Convert to string to ensure consistency
df_clean['city'] = df_clean['city'].astype(str)
df_clean['state'] = df_clean['state'].astype(str)

# --- 3. HANDLE MISSING VALUES ---

# A. Numerical columns - fill with median
numerical_cols = ['rating', 'review_count', 'price_per_night', 'tax', 'guest_rating']
for col in numerical_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# B. Star rating - keep as Unknown for missing values
df_clean['star_rating'] = df_clean['star_rating'].fillna(-1)  # Use -1 for Unknown
df_clean['star_rating_clean'] = df_clean['star_rating'].apply(
    lambda x: 'Unknown' if x == -1 else str(int(x))
)

# C. Categorical columns
categorical_cols = ['rating_description', 'nearest_landmark', 'price_range', 
                   'location', 'hotel_type', 'room_types']
for col in categorical_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna('Unknown')

# D. Distance column - fill missing with 'Na', not mean/median/high value
if 'Distance_to_Landmark(in_km)' in df_clean.columns:
    df_clean['Distance_to_Landmark(in_km)'] = df_clean['Distance_to_Landmark(in_km)'].apply(
        lambda x: x if pd.notnull(x) else -1     # since numerical colm keep it -1 for unknown values
    )


# E. Coordinates - fill with India center coordinates
if 'latitude' in df_clean.columns and 'longitude' in df_clean.columns:
    # Option 1: Create a flag for missing coordinates
    df_clean['has_coordinates'] = (~df_clean['latitude'].isna() & ~df_clean['longitude'].isna()).astype(int)
    
    # Option 2: Drop coordinate columns if too many missing values (>50%)
    lat_missing_pct = df_clean['latitude'].isna().mean()
    lon_missing_pct = df_clean['longitude'].isna().mean()
    
    if lat_missing_pct > 0.5 or lon_missing_pct > 0.5:
        print(f"Dropping coordinates due to high missingness: lat={lat_missing_pct:.1%}, lon={lon_missing_pct:.1%}")
        df_clean = df_clean.drop(columns=['latitude', 'longitude'])
    else:
        # If keeping coordinates, fill with median of available coordinates
        df_clean['latitude'] = df_clean['latitude'].fillna(df_clean['latitude'].median())
        df_clean['longitude'] = df_clean['longitude'].fillna(df_clean['longitude'].median())

# F. Handle amenities
if 'amenities' in df_clean.columns:
    df_clean['amenities'] = df_clean['amenities'].fillna('[]')
    
    # Process amenities more carefully
    def safe_eval_amenities(x):
        try:
            if isinstance(x, str) and x.startswith('['):
                return ast.literal_eval(x)
            return []
        except:
            return []
    
    df_clean['amenities_list'] = df_clean['amenities'].apply(safe_eval_amenities)
    
    # Get top 20 most common amenities only to avoid too many columns
    all_amenities = []
    for amenity_list in df_clean['amenities_list']:
        all_amenities.extend(amenity_list)
    
    from collections import Counter
    top_amenities = Counter(all_amenities).most_common(20)
    
    # Create binary columns for top amenities
    for amenity, count in top_amenities:
        col_name = f"amenity_{amenity.replace(' ', '_').replace('-', '_').lower()}"
        df_clean[col_name] = df_clean['amenities_list'].apply(
            lambda x: 1 if amenity in x else 0
        )
    
    # Count total amenities
    df_clean['total_amenities_count'] = df_clean['amenities_list'].apply(len)
    
    # Drop original amenities columns
    df_clean = df_clean.drop(columns=['amenities', 'amenities_list'])

# G. Description - convert to TF-IDF features (limited)
if 'description' in df_clean.columns:
    df_clean['description'] = df_clean['description'].fillna('')
    
    # Only create TF-IDF if we have descriptions
    if df_clean['description'].str.len().sum() > 0:
        tfidf = TfidfVectorizer(max_features=30, stop_words='english', min_df=5)
        description_tfidf = tfidf.fit_transform(df_clean['description'])
        
        # Add TF-IDF features
        feature_names = [f"desc_{name}" for name in tfidf.get_feature_names_out()]
        tfidf_df = pd.DataFrame(description_tfidf.toarray(), 
                               columns=feature_names, 
                               index=df_clean.index)
        df_clean = pd.concat([df_clean, tfidf_df], axis=1)
    
    df_clean = df_clean.drop(columns=['description'])

# H. Data quality score - keep as is if exists
if 'data_quality_score' in df_clean.columns:
    df_clean['data_quality_score'] = df_clean['data_quality_score'].fillna(df_clean['data_quality_score'].median())

# --- 4. ENCODE CATEGORICAL VARIABLES (SMART APPROACH) ---

# Create a dictionary to store label encoders
label_encoders = {}

# Label encoding for high cardinality
for col in high_cardinality_cols:
    if col in df_clean.columns:
        le = LabelEncoder()
        df_clean[f'{col}_encoded'] = le.fit_transform(df_clean[col].astype(str))
        
        # Save the encoder for later reference
        label_encoders[col] = le

# Now you can access the mappings
print("State encoding mapping:")
for i, state in enumerate(label_encoders['state'].classes_):
    print(f"{i}: {state}")

print("\nCity encoding mapping:")
for i, city in enumerate(label_encoders['city'].classes_):
    print(f"{i}: {city}")

print("\nHotel name encoding mapping (first 10):")
for i, hotel in enumerate(label_encoders['hotel_name'].classes_[:10]):
    print(f"{i}: {hotel}")

# For high cardinality categoricals like city/state, use label encoding
# For low cardinality ones, use one-hot encoding

high_cardinality_cols = ['city', 'state', 'hotel_name']
low_cardinality_cols = ['rating_description', 'nearest_landmark', 'price_range', 
                       'location', 'hotel_type', 'room_types', 'star_rating_clean']

# Label encoding for high cardinality
for col in high_cardinality_cols:
    if col in df_clean.columns:
        le = LabelEncoder()
        df_clean[f'{col}_encoded'] = le.fit_transform(df_clean[col].astype(str))

# One-hot encoding for low cardinality (limit categories)
for col in low_cardinality_cols:
    if col in df_clean.columns:
        # Limit to top 10 categories to avoid explosion
        top_categories = df_clean[col].value_counts().head(10).index
        df_clean[col] = df_clean[col].apply(
            lambda x: x if x in top_categories else 'Other'
        )
        
        # One-hot encode
        dummies = pd.get_dummies(df_clean[col], prefix=col)
        df_clean = pd.concat([df_clean, dummies], axis=1)

# Drop original categorical columns
categorical_to_drop = high_cardinality_cols + low_cardinality_cols + ['star_rating']
df_clean = df_clean.drop(columns=[col for col in categorical_to_drop if col in df_clean.columns])

# --- 5. SCALE NUMERICAL FEATURES ---
# Identify numerical columns for scaling
numerical_cols_to_scale = []
for col in df_clean.columns:
    if df_clean[col].dtype in ['float64', 'int64'] and not col.endswith('_encoded'):
        # Skip binary columns (0s and 1s)
        if not (df_clean[col].isin([0, 1]).all()):
            numerical_cols_to_scale.append(col)

print(f"Numerical columns to scale: {numerical_cols_to_scale}")

if numerical_cols_to_scale:
    scaler = StandardScaler()
    df_clean[numerical_cols_to_scale] = scaler.fit_transform(df_clean[numerical_cols_to_scale])

# --- 6. FINAL CLEANUP ---
# Remove any constant columns
df_clean = df_clean.loc[:, df_clean.nunique() > 1]

print(f"Final dataset shape: {df_clean.shape}")
print(f"Final columns: {len(df_clean.columns)}")

# Display info about the cleaned dataset
print("\nDataset Info:")
print(df_clean.info())

print(f"\nMissing values per column:")
missing_counts = df_clean.isnull().sum()
print(missing_counts[missing_counts > 0])

print(f"\nData types:")
print(df_clean.dtypes.value_counts())

# Save the preprocessed data
df_clean.to_csv('preprocessed_hotel_data_final.csv', index=False)
print("\nPreprocessed data saved as 'preprocessed_hotel_data_final.csv'")

# Show sample of the processed data
print(f"\nSample of processed data:")
print(df_clean.head())

Initial shape: (4632, 30)
Initial columns: ['hotel_name', 'rating', 'rating_description', 'review_count', 'star_rating', 'location', 'nearest_landmark', 'Distance_to_Landmark', 'Distance_to_Landmark(in_km)', 'price_per_night', 'tax', 'city', 'state', 'address', 'price_range', 'latitude', 'longitude', 'amenities', 'hotel_type', 'phone_number', 'email', 'website', 'booking_url', 'guest_rating', 'description', 'check_in_time', 'check_out_time', 'room_types', 'data_source', 'data_quality_score']
After dropping columns: (4632, 21)
State encoding mapping:
0: Arunachal Pradesh
1: Assam
2: Delhi
3: Goa
4: Gujarat
5: Haryana
6: Himachal Pradesh
7: Karnataka
8: Kerala
9: Madhya Pradesh
10: Maharashtra
11: Rajasthan
12: Tamil Nadu
13: Telangana
14: Uttar Pradesh
15: West Bengal

City encoding mapping:
0: Agra
1: Ahmedabad
2: Bangalore
3: Bhopal
4: Chennai
5: Dispur
6: Faridabad
7: Gandhinagar
8: Ghaziabad
9: Hyderabad
10: Indore
11: Itanagar
12: Jaipur
13: Kanpur
14: Kolkata
15: Lucknow
16: Mumba